In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler

In [2]:
random_seed = 137
torch.manual_seed(random_seed)
np.random.seed(random_seed)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

print(device)

cuda:0


In [3]:
# Early stopping class from https://github.com/Bjarten/early-stopping-pytorch
from SurvNODE.EarlyStopping import EarlyStopping
from SurvNODE.SurvNODE_x_ranking import *

In [4]:
def measures(odesurv,initial,x,Tstart,Tstop,From,To,trans,status, multiplier=1.,points=500):
    with torch.no_grad():
        time_grid = np.linspace(0, multiplier, points)
        pvec = torch.zeros((points,x.shape[0]))
        surv_ode = odesurv.predict(x,torch.from_numpy(np.linspace(0,multiplier,points)).float().to(x.device))
        pvec = torch.einsum("ilkj,k->ilj",(surv_ode[:,:,:,:],initial))[:,:,0].cpu()
        pvec = np.array(pvec.cpu().detach())
        surv_ode_df = pd.DataFrame(pvec)
        surv_ode_df.loc[:,"time"] = np.linspace(0,multiplier,points)
        surv_ode_df = surv_ode_df.set_index(["time"])
        ev_ode = EvalSurv(surv_ode_df, np.array(Tstop.cpu()), np.array(status.cpu()), censor_surv='km')
        conc = ev_ode.concordance_td('antolini')
        ibs = ev_ode.integrated_brier_score(time_grid)
        inbll = ev_ode.integrated_nbll(time_grid)
    return conc,ibs,inbll

In [5]:
from sklearn_pandas import DataFrameMapper
import pandas as pd

def make_dataloader(df,Tmax,batchsize):
#     cols_standardize = ['x0', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']
#     cols_leave = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']

#     standardize = [([col], StandardScaler()) for col in cols_standardize]
#     leave = [(col, None) for col in cols_leave]
#     x_mapper = DataFrameMapper(standardize + leave)
#     X = x_mapper.fit_transform(df).astype('float32')

    X = df[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']].values
    
    X = torch.from_numpy(X).float().to(device)
    T = torch.from_numpy(df[["duration"]].values).float().flatten().to(device)
    T = T/Tmax
    T[T==0] = 1e-8
    E = torch.from_numpy(df[["event"]].values).float().flatten().to(device)

    Tstart = torch.from_numpy(np.array([0 for i in range(T.shape[0])])).float().to(device)
    From = torch.tensor([1],device=device).repeat((T.shape))
    To = torch.tensor([2],device=device).repeat((T.shape))
    trans = torch.tensor([1],device=device).repeat((T.shape))

    dataset = TensorDataset(X,Tstart,T,From,To,trans,E)
    loader = DataLoader(dataset, batch_size=batchsize, shuffle=True)
    return loader

In [6]:
from sklearn.model_selection import train_test_split

def odesurv_manual_benchmark(df_train, df_test,config,name):
    torch.cuda.empty_cache()
    df_train, df_val = train_test_split(df_train, test_size=0.2, stratify=df_train.loc[:,"event"])
    
    Tmax = df_train["duration"].max()
    
    train_loader = make_dataloader(df_train,Tmax/config["multiplier"],int(len(df_train)*config["batch_size"]))
    val_loader = make_dataloader(df_val,Tmax/config["multiplier"],len(df_val))
    test_loader = make_dataloader(df_test,Tmax/config["multiplier"],len(df_test))
    
    num_in = 9
    num_latent = config["num_latent"]
    layers_encoder =  [config["encoder_neurons"]]*config["num_encoder_layers"]
    dropout_encoder = [config["encoder_dropout"]]*config["num_encoder_layers"]
    layers_odefunc1 =  [config["odefunc_neurons1"]]*config["num_odefunc_layers1"]
#     layers_odefunc2 =  [config["odefunc_neurons2"]]*config["num_odefunc_layers2"]

    trans_matrix = torch.tensor([[np.nan,1],[np.nan,np.nan]]).to(device)

    encoder = Encoder(num_in,num_latent,layers_encoder, dropout_encoder).to(device)
    odefunc = ODEFunc(trans_matrix,num_in,num_latent,layers_odefunc1,config["softplus_beta"]).to(device)
    block = ODEBlock(odefunc).to(device)
    odesurv = SurvNODE(block,encoder).to(device)

    optimizer = torch.optim.Adam(odesurv.parameters(), weight_decay = config["weight_decay"], lr=config["lr"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=config["scheduler_gamma"], patience=config["scheduler_epoch"])

    early_stopping = EarlyStopping(name=name,patience=config["patience"], verbose=True)
    t = tqdm(range(200))
    
        
    for i in t:
        odesurv.train()
        
        loss_all = 0
        for mini,ds in enumerate(train_loader):
            myloss,_,_ = loss(odesurv,*ds,mu=config["mu"])
            optimizer.zero_grad()
            myloss.backward()    
            optimizer.step()

            loss_all += myloss
        odesurv.eval()
        with torch.no_grad():
            lossval,conc,ibs,ibnll = 0., 0., 0., 0.
            for _,ds in enumerate(val_loader):
                t1,_,_ = loss(odesurv,*ds,mu=config["mu"])
                lossval += t1.item()
                t1,t2,t3 = measures(odesurv,torch.tensor([1.,0.],device=device),*ds,multiplier=config["multiplier"])
                conc += t1
                ibs += t2
                ibnll += t3
            early_stopping(lossval/len(val_loader), odesurv)
            scheduler.step(lossval/len(val_loader))
#             tune.report(score=lossval/len(val_loader), iterations=i)
            
            conc_test,ibs_test,ibnll_test = 0., 0., 0.
            print("it: "+str(i)+", train loss="+str(loss_all.item())+", validation loss="+str(lossval/len(val_loader))+", c="+str(conc/len(val_loader))+", ibs="+str(ibs/len(val_loader))+", ibnll="+str(ibnll/len(val_loader)))

        if early_stopping.early_stop:
            print("Early stopping")
            break
            t.refresh()
            t.set_postfix({"loss.:": myloss/len(train_loader)})

#     odesurv.load_state_dict(torch.load(name+'_checkpoint.pt'))

    odesurv.eval()
    with torch.no_grad():
        conc,ibs,ibnll = 0., 0., 0.
        for _,ds in enumerate(test_loader):
            t1,t2,t3 = measures(odesurv,torch.tensor([1.,0.],device=device),*ds,multiplier=config["multiplier"])
            conc += t1
            ibs += t2
            ibnll += t3
    return lossval/len(val_loader), conc/len(test_loader), ibs/len(test_loader), ibnll/len(test_loader)

In [7]:
from sklearn.model_selection import StratifiedKFold
from pycox import datasets

kfold = StratifiedKFold(5, shuffle=True)
df_all = datasets.metabric.read_df()
gen = kfold.split(df_all.iloc[:,df_all.columns.values!="event"],df_all.loc[:,"event"])

config = {'batch_size': 0.1, 
          'encoder_dropout': 0.0, 
          'encoder_neurons': 223, 
          'lr': 0.00005, 
          'mu': 0.0001, 
          'multiplier': 1.0, 
          'num_encoder_layers': 3, 
          'num_latent': 145, 
          'num_odefunc_layers1': 4, 
          'num_odefunc_layers2': 3, 
          'odefunc_neurons1': 1421, 
          'odefunc_neurons2': 1231, 
          'patience': 20, 
          'scheduler_epoch': 5, 
          'scheduler_gamma': 0.1, 
          'softplus_beta': 1.0, 
          'weight_decay': 0.0001}

odesurv_bench_vals = []
# for g in gen:
#     df_train = df_all.iloc[g[0]]
#     df_test =  df_all.iloc[g[1]]
#     _, conc, ibs, ibnll = odesurv_manual_benchmark(df_train,df_test,config,"metabric_test")
#     odesurv_bench_vals.append([conc,ibs,ibnll])
#     scores = torch.tensor(odesurv_bench_vals)
#     print(scores)
#     print(torch.mean(scores, dim=0))
#     print(torch.std(scores, dim=0))
    
# print(scores)
# print(torch.mean(scores, dim=0))
# print(torch.std(scores, dim=0))

In [8]:
# print("c="+str(np.mean(np.array(odesurv_bench_vals)[:,0]))+"+-"+str(np.std(np.array(odesurv_bench_vals)[:,0])))
# print("ibs="+str(np.mean(np.array(odesurv_bench_vals)[:,1]))+"+-"+str(np.std(np.array(odesurv_bench_vals)[:,1])))
# print("ibnll="+str(np.mean(np.array(odesurv_bench_vals)[:,2]))+"+-"+str(np.std(np.array(odesurv_bench_vals)[:,2])))

In [9]:
from pycox import datasets
df = datasets.metabric.read_df()
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df.loc[:,"event"])

In [10]:
torch.cuda.empty_cache()
from hyperopt import hp
args = {
    "lr": hp.choice("lr", [1e-4, 5e-4]),
    "weight_decay": hp.choice("weight_decay", [1e-3, 1e-4]),
    "num_latent": hp.randint('num_latent', 20, 200),
    "encoder_neurons": hp.randint('encoder_neurons', 100, 800),
    "num_encoder_layers": hp.randint("num_encoder_layers", 2, 5),
    "encoder_dropout": 0.,
    "odefunc_neurons1": hp.randint('odefunc_neurons1', 100, 1500),
    "num_odefunc_layers1": hp.randint("num_odefunc_layers1", 2, 5),
#     "odefunc_neurons2": hp.randint('odefunc_neurons2', 100, 1500),
#     "num_odefunc_layers2": 3,
    "batch_size": 128,
    "multiplier": 1.,
    "mu": 1e-4,
    "softplus_beta": hp.choice("softplus_beta", [1., 0.1]),
    "scheduler_epoch": 5,
    "scheduler_gamma": 0.1,
    "patience": 20
}
args = {'batch_size': hp.uniform('batch_size', 0.01, 0.5), 
          'encoder_dropout': hp.choice('encoder_dropout', [0.0,0.1,0.2]), 
          'encoder_neurons': hp.quniform('encoder_neurons', 20, 300), 
          'lr': 0.00005, #hp.choice('lr', [0.00005, 0.0005, 0.0001]), 
          'mu': 0.0001, 
          'multiplier': hp.choice('multiplier', [1.0, 2.0]), 
          'num_encoder_layers': 3, 
          'num_latent': hp.quniform('num_latent', 20, 200), 
          'num_odefunc_layers1': hp.randint('num_odefunc_layers1', 2, 5), 
#           'num_odefunc_layers2': 3, 
          'odefunc_neurons1': hp.quniform('odefunc_neurons1', 1000, 2000), 
#           'odefunc_neurons2': 1231, 
          'patience': 20, 
          'scheduler_epoch': 5, 
          'scheduler_gamma': 0.1, 
          'softplus_beta': 1.0, 
          'weight_decay': 0.0001}

In [15]:
# define an objective function
def objective(args):
    print(args)
    score, conc, ibs, ibnll = odesurv_manual_benchmark(df_train,df_test,args,"metabrick_test")
    print("\nResults: {}, {}, {}".format(conc, ibs, ibnll))
    with open("hp_sep_log.txt", "a") as f:
        f.write(str(args))
        f.write("\nResults: {}, {}, {}".format(conc, ibs, ibnll))
    return score

In [16]:
# # minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(objective, args, algo=tpe.suggest, max_evals=100)

{'batch_size': 128, 'encoder_dropout': 0.0, 'encoder_neurons': 730, 'lr': 0.0005, 'mu': 0.0001, 'multiplier': 1.0, 'num_encoder_layers': 3, 'num_latent': 129, 'num_odefunc_layers1': 2, 'odefunc_neurons1': 556, 'patience': 20, 'scheduler_epoch': 5, 'scheduler_gamma': 0.1, 'softplus_beta': 0.1, 'weight_decay': 0.0001}
  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/200 [00:00<?, ?it/s]

Validation loss decreased (inf --> 0.408804).  Saving model ...                                                        
it: 0, train loss=0.43715882301330566, validation loss=0.4088039994239807, c=0.5318760811448341, ibs=0.28881434103571474, ibnll=1.0297291634181962
Validation loss decreased (0.408804 --> 0.401936).  Saving model ...                                                   
it: 1, train loss=0.4290658235549927, validation loss=0.40193551778793335, c=0.5025947476018242, ibs=0.2871722229746581, ibnll=1.0169307097269589
Validation loss decreased (0.401936 --> 0.393154).  Saving model ...                                                   
it: 2, train loss=0.421892374753952, validation loss=0.39315417408943176, c=0.5269067463437648, ibs=0.2850657185358277, ibnll=1.000712252099843
Validation loss decreased (0.393154 --> 0.384757).  Saving model ...                                                   
it: 3, train loss=0.41279610991477966, validation loss=0.3847571909427643, c=0.5802

it: 30, train loss=0.224263533949852, validation loss=0.21048656105995178, c=0.40622739424437804, ibs=0.21577546153722205, ibnll=0.6358479137522306
Validation loss decreased (0.210487 --> 0.207347).  Saving model ...                                                   
it: 31, train loss=0.22070057690143585, validation loss=0.20734748244285583, c=0.4651360276773078, ibs=0.2137503811140742, ibnll=0.6287078515854274
Validation loss decreased (0.207347 --> 0.204393).  Saving model ...                                                   
it: 32, train loss=0.2173394113779068, validation loss=0.2043931931257248, c=0.5709073753734863, ibs=0.21178570043044265, ibnll=0.621923310872244
Validation loss decreased (0.204393 --> 0.201618).  Saving model ...                                                   
it: 33, train loss=0.21417075395584106, validation loss=0.20161780714988708, c=0.4938826859569115, ibs=0.2098826812735606, ibnll=0.6154839329448738
Validation loss decreased (0.201618 --> 0.199014).

Validation loss decreased (0.170263 --> 0.170001).  Saving model ...                                                   
it: 61, train loss=0.1764790266752243, validation loss=0.1700010597705841, c=0.515301147979242, ibs=0.18011765520098783, ibnll=0.5311793426095048
Validation loss decreased (0.170001 --> 0.169756).  Saving model ...                                                   
it: 62, train loss=0.17615236341953278, validation loss=0.1697555035352707, c=0.6110709231011165, ibs=0.1796914734476861, ibnll=0.5302094810620012
Validation loss decreased (0.169756 --> 0.169544).  Saving model ...                                                   
it: 63, train loss=0.17584918439388275, validation loss=0.1695440709590912, c=0.6051580437175657, ibs=0.17929978944836866, ibnll=0.5293374160920684
Validation loss decreased (0.169544 --> 0.169359).  Saving model ...                                                   
it: 64, train loss=0.17558042705059052, validation loss=0.16935916244983673, c=

...

it: 121, train loss=0.15417367219924927, validation loss=0.14972524344921112, c=0.6277068295391449, ibs=0.1595242786618832, ibnll=0.4742511024992986
EarlyStopping counter: 2 out of 20                                                                                     
it: 122, train loss=0.1540268510580063, validation loss=0.14912784099578857, c=0.6282003825035474, ibs=0.15953965774124437, ibnll=0.47374092574184623
Validation loss decreased (0.149099 --> 0.148885).  Saving model ...                                                   
it: 123, train loss=0.15377511084079742, validation loss=0.14888527989387512, c=0.6282312295638226, ibs=0.1595425126165235, ibnll=0.4735164909614056
EarlyStopping counter: 1 out of 20                                                                                     
it: 124, train loss=0.15363933145999908, validation loss=0.1494952142238617, c=0.6282929236843728, ibs=0.1594289276465864, ibnll=0.4738722307427349
Validation loss decreased (0.148885 --> 0.14

it: 151, train loss=0.14918789267539978, validation loss=0.14782275259494781, c=0.6225553704731939, ibs=0.1586143842031684, ibnll=0.4711341765510447
Validation loss decreased (0.146882 --> 0.146602).  Saving model ...                                                   
it: 152, train loss=0.1492001712322235, validation loss=0.14660190045833588, c=0.6246838176321796, ibs=0.15894757534690626, ibnll=0.47071134268931225
EarlyStopping counter: 1 out of 20                                                                                     
it: 153, train loss=0.1491970270872116, validation loss=0.14798963069915771, c=0.6220001233882411, ibs=0.15857005908651442, ibnll=0.47114315329778467
EarlyStopping counter: 2 out of 20                                                                                     
it: 154, train loss=0.14903947710990906, validation loss=0.14667101204395294, c=0.6240051823061262, ibs=0.15875647833098777, ibnll=0.47039928278947296
EarlyStopping counter: 3 out of 20      

job exception: CUDA out of memory. Tried to allocate 862.00 MiB (GPU 0; 8.00 GiB total capacity; 5.17 GiB already allocated; 342.27 MiB free; 6.11 GiB reserved in total by PyTorch)



  8%|███                                   | 8/100 [16:35:00<190:42:37, 7462.58s/trial, best loss: 0.13281087577342987]


RuntimeError: CUDA out of memory. Tried to allocate 862.00 MiB (GPU 0; 8.00 GiB total capacity; 5.17 GiB already allocated; 342.27 MiB free; 6.11 GiB reserved in total by PyTorch)

In [ ]:
print(best)
print(space_eval(space, best))